This scripts gets as input two csv files (ref and hyp) with the following structure:

    word    begin   end
    ik      0.2     0.3
    begin   0.3     0.35
    etc.

JASMIN reference files are created here: fluency-features-using-ASR/ref_fluency_scripts/01-pace-phrasing-from-tg.ipynb
JASMIN ASR files are created here: 

In [ ]:
import os
import pandas as pd
import glob
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
def computeNrTPs(asrDF, otDF, collar):

    TP_counter = 0

    for idx, row in asrDF.iterrows():

        start = row['start']
        end = row['end']
        textNorm = row['text_norm']

        acceptable_start_begintime = start - collar
        acceptable_end_begintime = start + collar
        acceptable_start_endtime = end - collar
        acceptable_end_endtime = end + collar

        # Select all rows in otDF that fall within the acceptable_start collar and acceptable_end collar
        otDF_sel = otDF[(otDF['start'] >= acceptable_start_begintime) & (otDF['start'] <= acceptable_end_begintime) & (otDF['end'] >= acceptable_start_endtime) & (otDF['end'] <= acceptable_end_endtime) & (otDF['text_norm'] == textNorm)]

        if len(otDF_sel) > 0:
            TP_counter += 1

    return TP_counter

In [ ]:
def computeMetrics(asrDfFiles, collar, outputDirMetrics, otDfDir):

    outputList = []

    for asrDfFile in asrDfFiles:
        # Get basename
        basename = os.path.basename(asrDfFile).replace('.tsv', '')

        # Get corresponding orth. trans. file
        otDfFile = os.path.join(otDfDir, basename + '.tsv')
        assert os.path.exists(otDfFile), 'OT file ' + basename + ' does not exist.'

        # Read both files
        asrDF = pd.read_csv(asrDfFile, index_col = 0, sep='\t')
        otDF = pd.read_csv(otDfFile, index_col = 0, sep='\t')

        # PREPROCESS FOR COMPARISON
        otDF = otDF.rename(columns = {'start_time': 'start', 'end_time': 'end'})

        # Compute Precision (DF1 = asrDF)
        TP_counter = computeNrTPs(asrDF, otDF, collar)

        precision = round(TP_counter / len(asrDF), 4)

        recall = round(TP_counter /len (otDF), 4)

        try:

            F1 = round((2*precision*recall)/(precision+recall), 4)

        except:
            F1 = np.nan

        outputList.append([basename, collar, precision, recall, F1, TP_counter, len(asrDF), len(otDF)])

    outputDF = pd.DataFrame(outputList, columns= ['audioID', 'collar', 'precision', 'recall', 'F1', 'TP_counter', 'lenAsrDF', 'lenOtDF'])
    outputDF.to_csv(os.path.join(outputDirMetrics, 'test_metrics_'+str(collar)+'.tsv'), sep='\t')

    return outputDF['F1'].mean(), outputDF['F1'].std(), outputDF['precision'].mean(), outputDF['precision'].std(), outputDF['recall'].mean(), outputDF['recall'].std(), outputDF

In [ ]:
basePath = '/vol/tensusers2/wharmsen/JASMIN-fluency-features/comp-q-read_nl_age7-11_nat-new'
# basePath = '/vol/tensusers2/wharmsen/JASMIN-fluency-features/comp-q-read_vl_age7-11_nat-new'
corpus = 'jasmin'

# basePath = '/vol/tensusers2/wharmsen/SERDA-fluency-features/comp1-new'
# corpus = 'serda'

recordingsPath = os.path.join(basePath, '03_metadata/recordingsDF.tsv')
recordingsDF = pd.read_csv(recordingsPath, index_col=0, sep='\t')

asrSystems = ['whispert', 'whispert_dis', 'whispert_vad_dis', 'whispert_prompts']

metricsOverviewList = []
metricsFileDFList = []
for asrSystem in asrSystems:

    """
    Define input paths
    """
    asrDfDir = os.path.join(basePath, '04_asr/' + asrSystem + '/timestamps-words')
    otDfDir = os.path.join(basePath, '06_manual_fluency_features/json-orth-trans')
    
    """
    Define output paths
    """
    outputDirMetrics = os.path.join(basePath, '04_asr/' + asrSystem + '/report_time_eval')
    if not os.path.exists(outputDirMetrics):
        os.makedirs(outputDirMetrics)


    """
    Perform the analysis
    """

    if corpus == 'jasmin':
        asrDfFiles = glob.glob(os.path.join(asrDfDir, '*story[1-2].tsv'))
    elif corpus == 'serda':
        asrDfFiles = glob.glob(os.path.join(asrDfDir, '*.tsv'))
    print(len(asrDfFiles), 'asrDF files', os.path.join(asrDfDir, '*.tsv'))

    for collar in np.arange(0.2, 0.4, 0.1):
        F1_mean, F1_std, Prec_mean, Prec_std, Recall_mean, Recall_std, metricsFileDF = computeMetrics(asrDfFiles, collar, outputDirMetrics, otDfDir)
        metricsOverviewList.append([asrSystem, collar, F1_mean, F1_std, Prec_mean, Prec_std, Recall_mean, Recall_std])
        metricsFileDF['asrSystem'] = [asrSystem] * len(metricsFileDF)
        metricsFileDFList.append(metricsFileDF)

metricsOverviewDF = pd.DataFrame(metricsOverviewList, columns=['asrSystem', 'collar','F1_mean', 'F1_std', 'Prec_mean', 'Prec_std', 'Recall_mean', 'Recall_std'])
metricsOverviewDF.to_csv(os.path.join(basePath, '04_asr/timingAccMetricsOverviewDF.tsv'), sep='\t')

metricsDF = pd.concat(metricsFileDFList)
metricsDF.to_csv(os.path.join(basePath, '04_asr/timingAccMetricsDF.tsv'), sep='\t')